In [1]:
import lxml
import codecs
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import glob

In [11]:
temp=pd.read_pickle("/home/harsh.d/Shoaib/arxiv_metadata/metadata_ids.pkl")

In [12]:
temp

,,id
title,authors_tup,
"structadmm: a systematic, high-efficiency framework of structured weight pruning for dnns","(t zhang, s ye, k zhang, x ma, n liu, l zhang, j tang, k ma, x lin, m fardad, y wang)",0
distributed optimization and statistical learning via the alternating direction method of multipliers,"([ references, boyd)",1
learning both weights and connections for efficient neural network,(),2
convergence analysis of alternating direction method of multipliers for a family of nonconvex problems,"(m luo, z hong, ; luo, hong)",3
learning structured sparsity in deep neural networks,(),4
...,...,...
l1-depth revisited: a robust angle-based outlier factor in high-dimensional space,"(n pham,)",62841251
locality adaptive discriminant analysis framework,"(x li, q wang, f nie, m chen)",62841252
linear discriminant analysis: new formulations and overfit analysis,"(d luo, c ding, h huang)",62841254


# Paper ID - Reference ID mappings

In [13]:
#convert dataframe to dictionary
temp=temp.to_dict('index')

In [2]:
xml_file_new = glob.glob("/home/harsh.d/arxiv/xml_new/*")
xml_file_old = glob.glob("/home/harsh.d/old_xml_files/*")
xml_files = xml_file_old + xml_file_new

In [49]:
snap_df=pd.read_json("/home/harsh.d/Shoaib/arxiv_metadata/arxiv-metadata-oai-snapshot.json",lines=True)
snap_df['id']=snap_df['id'].str.replace('/','')
snap_df['title']=snap_df['title'].str.lower()

In [50]:
"""Get Data with Citation-Reference Mapping"""

def get_data(file):
    
    data={}
    names=[]

    # Read XML
    with codecs.open(file,'r',"utf-8") as tei:
        soup = BeautifulSoup(tei, 'lxml')

    # Paper Title ID 
    title=soup.teiheader.title.get_text().lower()
    if(title==''):
        title=snap_df[snap_df['id']==file.split('/')[-1][:-8]]['title'].values[0].lower()

    # Paper Author Details    
    for author in soup.teiheader.find_all('author'):

        if(author.find('persname')!=None):

            if(author.find('persname').forename!=None and author.find('persname').surname!=None):
                names.append(author.find('persname').forename.get_text()[0].lower()+" "+author.find('persname').surname.get_text().lower())

            elif(author.find('persname').forename==None):
                names.append(author.find('persname').surname.get_text().lower())

            else:
                names.append(author.find('persname').forename.get_text()[0].lower())
    
    try:
        paper_id=temp[(title,tuple(names))]['id']
    except:
        paper_id=''
                
    # Dictionary containg <XML ID: Citation> format
    citations={}
    for targets in soup.find_all('ref'):
        if(targets.get('type')=='bibr' and targets.get('target')!=None):
            citations[targets.get('target')]=targets.get_text().lower()
    
    # Extract Citation, Reference Titles, Reference Authors 
    for ref in soup.back.find_all('biblstruct'):
        
        if('#'+ref.get("xml:id") in citations):
            
            cit=citations['#'+ref.get("xml:id")]
            
#             xml_id=ref.get("xml:id")
            
            for t in ref.find_all('title'):
                if(t.get_text()!=''):
                    ref_title=t.get_text().lower()
                    break
            
#             if(ref.find(type="arXiv")!=None):
#                 ref_doi=ref.find(type="arXiv").get_text().lower()
#             elif(ref.find(type="DOI")!=None):
#                 ref_doi=ref.find(type="DOI").get_text().lower()
#             else:
#                 ref_doi=''
            
            ref_authors=[]
            for author in ref.find_all('author'):

                if(author.find('persname')!=None):

                    if(author.find('persname').forename!=None and author.find('persname').surname!=None):
                        ref_authors.append(author.find('persname').forename.get_text()[0].lower()+" "+author.find('persname').surname.get_text().lower())

                    elif(author.find('persname').forename==None):
                        ref_authors.append(author.find('persname').surname.get_text().lower())

                    else:
                        ref_authors.append(author.find('persname').forename.get_text()[0].lower())
                        
#             titles.append(ref_title)
#             authors.append(ref_authors)
#             doi.append(ref_doi)
            try:
                ref_paper_id=temp[(ref_title,tuple(ref_authors))]['id']
            except:
                ref_paper_id=''
        
#             data[xml_id]={
#               'citation':cit,
#               'reference':ref_title,
#               'authors':ref_authors
#             }

            data[cit]=ref_paper_id
            
        
    return file.split('/')[-1],paper_id,data

In [51]:
import multiprocessing
mapping_outputs=[]

def driver_func():
    
    # Note: Keep Num_Process <= 60 to avoid system hang
    PROCESSES = 40
    
    with multiprocessing.Pool(PROCESSES) as pool:
        results=[pool.apply_async(get_data, args=(p,)) for p in tqdm(xml_files[:100000])]
         
        for r in tqdm(results):
            try:
                if(r.get(timeout=5)!=None):
                    mapping_outputs.append(r.get())
                
            except:
                pass

        pool.close()
        pool.join()

driver_func()

In [78]:
import multiprocessing
mapping_outputs=[]

def driver_func():
    
    # Note: Keep Num_Process <= 60 to avoid system hang
    PROCESSES = 40
    
    with multiprocessing.Pool(PROCESSES) as pool:
        results=[pool.apply_async(get_data, args=(p,)) for p in tqdm(xml_files)]
         
        for r in tqdm(results):
            try:
                if(r.get(timeout=5)!=None):
                    mapping_outputs.append(r.get())
                
            except:
                pass

        pool.close()
        pool.join()

driver_func()

In [80]:
cit_ref=pd.DataFrame(mapping_outputs)

In [81]:
cit_ref

,0,1,2
0,1807.11091.tei.xml,0,"{'[boyd et al., 2011;': 1, 'han et al., 2016]'..."
1,hep-ph0003173.tei.xml,6,"{'[1]': 7, '2': 8, '[3]': 9, '[5]': 10, '[6]':..."
2,2002.06870.tei.xml,22,"{'(1)': 23, '(2)': 24, '(3)': 25, '[4,': 26, '..."
3,1204.6600.tei.xml,45,"{'[1]': 46, '[2]': 47, '[3]': 48, '[4]': 49, '..."
4,1404.3331.tei.xml,99,"{', aldous, 1985': 100, ', bertoin, 2006': 101..."
...,...,...,...
1940483,2011.02495.tei.xml,62841089,"{'abbott et al. 2009': 856898, '(becker 2015)'..."
1940484,2101.06890.tei.xml,52360140,"{'[1]': 2977897, '[2,': 6792658, '[3]': 628411..."
1940485,2112.01060.tei.xml,62841141,"{'[2]': 62841142, '[3]': 2259011, '[4,': 21118..."
1940486,2106.06977.tei.xml,62841167,"{'[1]': 10882, '[2]': 146279, '[3]': 4296414, ..."
